In [37]:
from dotenv import load_dotenv
load_dotenv()
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import os
import json


pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

pc.create_index(
  name="rag",
  dimension=4096,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
    ) 
)

In [38]:
data = json.load(open("reviews.json"))

In [39]:
processed_data = []
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    )
completion = client.chat.completions.create(
  
  model="meta-llama/llama-3-8b-instruct:free",
  messages=[
    {
      "role": "user",
      "content": "Say this is a test",
    },
  ],
)
print(completion.choices[0].message.content)
for review in data['reviews']:
    response = client.embeddings.create(
        input = review['review'],
        model = "text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })
    index = pc.Index("rag")
    upsert_response = index.upsert(
        vectors=processed_data,
        namespace="ns1",
    )
    print(f"Upserted count: {upsert_response['upserted_count']}")

    # Print index statistics
    print(index.describe_index_stats())

IT LOOKS LIKE WE HAVE A LIVE TEST ON OUR HANDS!

In that case, let's see how we do. I'll respond as expected, and we can see if my responses are correct or not. Got it?


APIStatusError: Error code: 405